<a href="https://colab.research.google.com/github/ishadvay3928/Video-Game-Sales-and-Engagement-Analysis-Project/blob/main/Video_Game_Sales_and_Engagement_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Project Name**    - **Video Game Sales and Engagement Analysis**



# **GitHub Link -**

https://github.com/ishadvay3928/Video-Game-Sales-and-Engagement-Analysis-Project/blob/main/Video_Game_Sales_and_Engagement_Analysis.ipynb

# ***Let's Begin !***

## ***1. Know Your Data***

### Import Libraries

In [ ]:
# Import Libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

### Dataset Loading

In [ ]:
# Load all the  Datasets

df_games = pd.read_csv("/content/games.csv")
df_vgsales = pd.read_csv("/content/vgsales.csv")

### Dataset First View

In [ ]:
# First Look
df_games.head()

In [ ]:
df_vgsales.head()

### Dataset Rows & Columns count

In [ ]:
# Dataset Rows & Columns count
df_games.shape

In [ ]:
df_vgsales.shape

### Dataset Information

In [ ]:
# Dataset Info
df_games.info()

In [ ]:
df_vgsales.info()

#### Duplicate Values

In [ ]:
# Duplicate Value Count
df_games.duplicated().sum()

In [ ]:
df_vgsales.duplicated().sum()

#### Missing Values/Null Values

In [ ]:
# Missing Values/Null Values Count of datasets
df_games.isnull().sum()

In [ ]:
df_vgsales.isnull().sum()

### What did you know about your dataset?

**games dataset**
- There are 1512 rows and 14 columns in the dataset.
- Rating have 13 missing Values followed by Team and
Summary having only 1 missing value each.


**vgsales dataset**
- There are 16598 rows and 11 columns in the dataset.
- Year have	271 missing Values and Publisher have 58 missing values.

## ***2. Understanding Your Variables***

In [ ]:
# Dataset Columns
df_games.columns

In [ ]:
df_vgsales.columns

In [ ]:
#Dataset Describe
df_games.describe(include='all')

In [ ]:
df_vgsales.describe(include='all')

### Variables Description

**1. games.csv (Game Engagement Data)**

* **Title:** Game name.
* **Rating:** User review score (numeric).
* **Genres:** Game categories (can be multiple).
* **Plays**: Number of playthroughs.
* **Backlogs**: Number of users who plan to play it.
* **Wishlist**: Number of users who wishlisted the game.
* **Release Date**: Date of game release
* **Platform**: Platform where game is available
* **Team (Developer)**: Developer of the game


**2. vgsales.csv (Sales Data)**

* **Name**: Game name.
* **Platform**: Console or device.
* **Year**: Year of release.
* **Genre**: Main category.
* **Publisher**: Game publisher.
* **NA_Sales**: Sales in North America
* **EU_Sales**: Sales in Europe.
* **JP_Sales**: Sales in Japan.
* **Other_Sales**: Sales in the rest of the world
* **Global_Sales**: Sales by region.

### Check Unique Values for each variable.

In [ ]:
# Check Unique Values for each variable of dataset.
df_games.nunique()

In [ ]:
df_vgsales.nunique()

## ***3. Data Wrangling***

### Data Wrangling Code

In [ ]:
# --- Wrangling vgsales.csv ---

# 1. Handle missing 'Year' and convert to Int64 (integer type that supports NaN)
median_year = df_vgsales['Year'].median()
df_vgsales['Year'] = df_vgsales['Year'].fillna(median_year).astype('Int64')

# 2. Handle missing 'Publisher'
df_vgsales['Publisher'].fillna('Unknown', inplace=True)

In [ ]:
# Save the wrangled vgsales data
df_vgsales.to_csv('vgsales_clean.csv', index=False)

In [ ]:
# --- Wrangling games.csv ---

# 1. Drop the unnecessary index column
df_games.drop(columns=['Unnamed: 0'], inplace=True)

In [ ]:
# 2. Convert 'Release Date' to datetime
df_games['Release Date'] = pd.to_datetime(df_games['Release Date'], format='%b %d, %Y', errors='coerce')

In [ ]:
# 3. Parse string lists for 'Team', 'Genres', and 'Reviews'
import ast
list_cols = ['Team', 'Genres', 'Reviews']
for col in list_cols:
    # Use ast.literal_eval for safe conversion of string-list to actual list
    df_games[col] = df_games[col].apply(lambda x: ast.literal_eval(x) if pd.notna(x) and isinstance(x, str) and x.startswith('[') and x.endswith(']') else x)

In [ ]:
# 4. Function to convert K-suffixed strings (e.g., '3.9K') to numbers
def convert_k_suffix(value):
    if isinstance(value, str):
        value = value.strip().replace(',', '')
        if 'K' in value:
            return int(float(value.replace('K', '')) * 1000)
        try:
            return int(value)
        except ValueError:
            return np.nan
    elif pd.isna(value):
        return np.nan
    return value

# Apply the conversion to all K-suffixed columns
k_cols = ['Times Listed', 'Number of Reviews', 'Plays', 'Playing', 'Backlogs', 'Wishlist']
for col in k_cols:
    df_games[col] = df_games[col].apply(convert_k_suffix).astype('Int64')

In [ ]:
# 5. Handle remaining missing values
# Fill missing 'Rating' with median
median_rating = df_games['Rating'].median()
df_games['Rating'].fillna(median_rating, inplace=True)

# Fill missing 'Summary'
df_games['Summary'].fillna('No Summary', inplace=True)

# Fill remaining missing 'Release Date' (the NaT values from step 2) with the median date
df_games['Release Date'].fillna(df_games['Release Date'].median(), inplace=True)

# Fill remaining missing 'Team' with placeholder
df_games['Team'].fillna('Unknown Team', inplace=True)

In [ ]:
# Save the wrangled games data
df_games.to_csv('games_clean.csv', index=False)

### What all manipulations have you done and insights you found?

#### **Key Manipulations:**

* **Handled Missing Values in `vgsales.csv`:**

  * Filled missing `Year` values with the median year and converted the column to `Int64` type to allow integers with NaN support.
  * Replaced missing `Publisher` entries with `"Unknown"`.

* **Cleaned `games.csv`:**

  * Dropped the redundant `Unnamed: 0` index column.
  * Converted `Release Date` into proper datetime format, coercing unparseable strings into `NaT`.
  * Parsed stringified lists (`Team`, `Genres`, `Reviews`) into actual Python lists using `ast.literal_eval`.
  * Created and applied a function to convert "K"-suffixed values (e.g., `"3.9K"`) into numeric integers across columns (`Times Listed`, `Number of Reviews`, `Plays`, `Playing`, `Backlogs`, `Wishlist`).
  * Converted these numeric columns into `Int64` to maintain integer representation with NaN support.
  * Filled missing values:

    * `Rating` with the median rating.
    * `Summary` with `"No Summary"`.
    * `Release Date` NaT entries with the median release date.
    * `Team` with `"Unknown Team"`.

* **Saved Cleaned Data:**
  Exported cleaned versions of both datasets (`vgsales_wrangled.csv` and `games_wrangled_final.csv`) for reproducibility and further analysis.


---


#### **Insights Gained:**

* Handling missing values with domain-appropriate replacements (median for numeric, placeholders for categorical/text) ensures that analyses remain consistent and no rows are unnecessarily dropped.
* Converting release dates into proper datetime format enables temporal trend analysis, such as release frequency by year, seasonal patterns, or correlation with ratings/reviews.
* Transforming stringified lists into actual Python lists allows easier querying, filtering, and analysis of genres, teams, and review content.
* Standardizing numeric fields (like "K"-suffixed values) improves accuracy in aggregation, comparison, and statistical analysis.
* Saving wrangled datasets provides a reliable baseline for downstream analytics such as sales trends, publisher performance, or relationships between ratings, reviews, and player engagement.
